In [1]:
import tensorflow as tf
import cv2
import mediapipe as mp
import numpy as np
import os

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

In [2]:
imgs = []
path = os.listdir('images')

for idx,i in enumerate(path):
    i = cv2.imread(os.path.join('images',path[idx]))
    i = cv2.resize(i, (320,570))
    imgs.append(i)
    

In [3]:
loaded_model = tf.keras.models.load_model("models/my_model.h5")

classes = ["Aki Hayakawa", "Gojo Satoru", "Rock Lee"]

In [4]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()

h,w,_ = frame.shape
pad = 30

while True:
    ret, frame = cap.read()

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    
    k = cv2.waitKey(1)

    if results.multi_hand_landmarks:
        img = frame.copy()
        # Loop through each detected hand
        x_min, y_min, x_max, y_max = float('inf'), float('inf'), -float('inf'), -float('inf')
        
        for hand_landmarks in results.multi_hand_landmarks:
            # Extract hand bounding box coordinates
            for landmark in hand_landmarks.landmark:
                x, y, _ = int(landmark.x * w), int(landmark.y * h), landmark.z
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)

        # Crop the hand from the frame
        hand_crop = img[y_min-pad:y_max+pad,x_min-pad:x_max+pad]
        
        # Resize the frame
        if 0 not in hand_crop.shape:
    
            resized_hand = cv2.resize(hand_crop, (250, 250))
            image = np.expand_dims(resized_hand, axis=0)
        
            class_probabilities = loaded_model.predict(image, verbose=False)

            predicted_class = np.argmax(class_probabilities)
            
            # Display the predicted class label on the frame
            class_label = "Character: " + str(classes[predicted_class])
            cv2.putText(frame, class_label, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
#             cv2.imshow('Cropped Hand', resized_hand)
            cv2.imshow('Character', imgs[predicted_class])    
    else:
        cv2.putText(frame, 'Call your Character', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    # Display the resized frame
    cv2.imshow('Hand Detection', frame)

    # Exit the loop when 'q' is pressed
    if k == 27:
        break

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [5]:
# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()